In [ ]:
import requests
import re

In [ ]:
class stationDataHandler():
    def __init__(self):
        self.baseUrl='http://www.cehq.gouv.qc.ca/suivihydro/'
    def getStations(self):
        """Fetches all station IDs
        """
        stationRegex = re.compile('.*option value="[0-9]')
        self.stations=[]
        try:
            source=requests.get('%sdefault.asp' % (self.baseUrl))
            for line in source.iter_lines():
                if line.isascii() and re.match(stationRegex, line.decode('utf-8')):
                    station=line.decode('utf-8').split('"')[1].strip()
                    #This is just to avoid dupes
                    if station not in self.stations:
                        self.stations.append(station)
                    else:
                        print('Duplicate station ID found: {}'.format(station))
        except:
            print("Unexpected error:", sys.exc_info()[0])
            raise
    def stationMetaFetch(self, station):
        """Function to gather the details of the station
        
        station - int - ID of the station
        """
        #TODO: everything
        print('should do this')

    def stationDataFetch(self, station):
        source=requests.get('{}fichier_donnees.asp?NoStation={}'.format(self.baseUrl, station))
        stationData=[]
        for line in source.iter_lines():
            if line.startswith(b'D'):
                columns=line.replace(b'\xe9', b'e').decode('utf-8').split()
            elif not line.startswith(b'D') and not line == b'' and not line.startswith(b'A'):
                #nLine=line.replace(b'\xa0', b'').decode('utf-8').replace('\t', ' ')
                #stationData.append('{}\n'.format(nLine.replace(',', '.')))
                nLine=line.replace(b'\xa0', b'').decode('utf-8').replace(',', '.').split()
                stationData.append(' '.join(map(lambda x: '[{}]{}'.format(x[0], x[1]), zip(columns, nLine))))
        return stationData
    def writeStationData(self, station):
        data=self.stationDataFetch(station)
        if station not in self.stations:
            #TODO: should instead try to fetch it, if not real then error
            raise Exception('station id has not been discovered, likely wrong number')
        
        with open('./data/{}'.format(station), 'w') as file:
            for line in data:
                file.write('{}\n'.format(line))

In [ ]:
o=stationDataHandler()
o.getStations()
#print(o.stations)
for station in o.stations:
    o.writeStationData(station)
#o.writeStationData('043301')

In [ ]:
data = requests.get('http://www.cehq.gouv.qc.ca/suivihydro/fichier_donnees.asp?NoStation=010802')

In [ ]:
for line in data.iter_lines():
    if line.startswith(b'D'):
        columns=line.replace(b'\xe9', b'e').decode('utf-8').split()
    elif not line == b'':
        nLine=line.replace(b'\xa0', b'').decode('utf-8').replace(',', '.').split()
        print(' '.join(map(lambda x: '[{}]{}'.format(x[0], x[1]), zip(columns, nLine))))

In [ ]:
#print(columns)
#print({columns[i]:nLine[i].replace(',', '.') for i in range(len(columns))})
#print([nLine[i].replace(',', '.') for i in range(len(columns))])

' '.join(map(lambda x: '[{}]{}'.format(x[0], x[1]), zip(columns, nLine)))